In the following notebook, I will be modeling consumer behavior in relation to energy demand. I am hoping to find specific patterns of energy use on a daily basis.

## Prepare Notebook

**Read in libraries**

In [5]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt

**Read in data**

In [6]:
#Set path to data
path = r'C:\Users\kishe\Documents\Data Science\Projects\Python Projects\In Progress\Spain Hourly Energy Demand and Weather\Data\02_Cleaned_Data'

#Read in data
df = pd.read_csv(path + '/2020_0620_Weather_Energy.csv')

**Preview data**

In [7]:
display(df.head())

,date_time,forecast_solar_day_ahead,forecast_wind_onshore_day_ahead,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,...,generation_other,generation_other_renewable,generation_solar,generation_waste,generation_wind_onshore,price_actual,price_day_ahead,temp,total_load_actual,total_load_forecast
0,2015-01-01 00:00:00,17.0,6436.0,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,...,43.0,73.0,49.0,196.0,6378.0,65.41,50.10,30.814633,25385.0,26118.0
1,2015-01-01 01:00:00,16.0,5856.0,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,...,43.0,71.0,50.0,195.0,5890.0,64.92,48.10,30.852860,24382.0,24934.0
2,2015-01-01 02:00:00,8.0,5454.0,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,...,43.0,73.0,50.0,196.0,5461.0,64.48,47.33,30.108448,22734.0,23515.0
3,2015-01-01 03:00:00,2.0,5151.0,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,...,43.0,75.0,50.0,191.0,5238.0,59.32,42.27,30.091044,21286.0,22642.0
4,2015-01-01 04:00:00,9.0,4861.0,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,...,43.0,74.0,42.0,189.0,4935.0,56.04,38.41,30.192620,20264.0,21785.0


# Clustering electricity usage profiles with K-means

**Prepare data**

In [8]:
df.date_time

0        2015-01-01 00:00:00
1        2015-01-01 01:00:00
2        2015-01-01 02:00:00
3        2015-01-01 03:00:00
4        2015-01-01 04:00:00
                ...         
35065    2018-12-31 19:00:00
35066    2018-12-31 20:00:00
35067    2018-12-31 21:00:00
35068    2018-12-31 22:00:00
35069    2018-12-31 23:00:00
Name: date_time, Length: 35070, dtype: object